## Record
A record is a convenient way to represent aggregate of values. A data `Server` containing a host and a port can be represented using: 

In [ ]:
public record Server(String host, int port) {}

This is equivalent to creating a class as:

In [ ]:
class Server {
    private final String host;
    private final int port;

    Server(String host, int port) {
        this.host = host;
        this.port = port;
    }

    String host() { return host; }
    int port() { return port; }

    public boolean equals(Object o) {
        if (!(o instanceof Server)) return false;
        Server other = (Server) o;
        return Objects.equals(other.host, host) && other.port == port;
    }

    public int hashCode() {
        return Objects.hash(host, port);
    }

    public String toString() {
        return String.format("Point[host=%d, y=%d]", host, port);
    }
}

Records represent immutable data carriers. The syntax of record therefore is chosen such that it contains list of all the states in the header. With the short syntax, the compiler automatically generates:
- a `public` accessor method with the same name and return type as the component, and a `private final` field with the same type as the component
- a *cononical constructor* whose signature is the same as the header
- `equals` and `hashCode` methods which ensure that two record values are equal if they are of the same type and contain equal component values
- `toString` method

**Record Restrictions:**
- A record class is implicitly final and therefore cannot be extended.
- It implicitly extends `java.lang.Record`, therefore it cannot extend another class.
- It cannot be `abstract`
- A record class cannot explicitly declare instance fields, and cannot contain instance initializers.
- A nested record class is implicitly `static`. Why? Because a non-static nested class by default contains reference to the instance of enclosing class. We don't want a record to contain anything more than what the record header advertises.

A record class can:
- be generic
- can have `static` fields, methods and initializers
- can declare instnce methods
- can implement interfaces
- can declare nested types
- header components can be annotated, the annotations are passed on to the fields, methods and constructors as per the annoatation targets. Propagation does not happen if accessor method and canonical constructor is explicitly provided (and don't specify the annotations).

## Constructor
A record class imnplicitly contains a constructor as shown in the example above. However, we can also explicitly provide the *canonical constructor* as:

In [ ]:
public record Server(String host, int port) {
    // Access modifier should atleast be that of the record, so
    // in this example, constructor cannot be package protected
    public Server {
        if (port < 0 || port > 65536)
            throw new IllegalArgumentException("Invalid port: " + port);

        host = host.trim();
    }
}

In the above compact constructor, the compiler automatically adds the `this.x = x;` statements. The compact form helps developers focus on validating and normalizing parameters without the tedious work of assigning parameters to fields. The above compact constructor is equivalent to:

In [ ]:
public Server(String host, int port) {
    if (port < 0 || port > 65536)
        throw new IllegalArgumentException("Invalid port: " + port);

    host = host.trim();
    
    this.host = host;
    this.port = port;
}

We can also declare additional constructors or use the non-compact form to define canonical constructor:

In [ ]:
public record Server(String host, int port) {
    // Using the non-compact form to define canonical constructor, this
    // must contain this.x = x statements
    public Server(String host, int port) {
        if (port < 0 || port > 65536)
            throw new IllegalArgumentException("Invalid port: " + port);
        
        this.host = host;
        this.port = port;
    }

    // Doesn't have to be public    
    public Server(int port) {
        this("localhost", port);
    }
}

## Local Record
A local record is implicitly static (as any nested record). This is different from local classes which cannot be static. Thus a local record cannot access local variables of the enclosing method.

## Accessing State
As stated earlier, a record class automatically creates accessors for each state variable defined in the record header. We can also override this by specifying our own accessors:

In [ ]:
// The example demostrates techniques to ensure immutability
record Province (String name, List<String> cities) {
    Province {
        cities = List.copyOf(cities); // ensures that outside code doesn't alter record's state
    }

    // Accessors must be public
    public List<String> cities() {
        return List.copyOf(cities);
    }
}